# Popularity Recomendation Model.

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline


/usr/local/Cellar/jupyterlab/2.0.1/libexec/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
columns=['userId', 'productId', 'userName','helpful','reviewtext','ratings','summary','reviewtime','rt']
beauty_df=pd.read_csv('/Users/sathyasripasham/Downloads/reviews.csv',names=columns)

In [6]:
beauty_df.head()

,userId,productId,userName,helpful,reviewtext,ratings,summary,reviewtime,rt
0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
1,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1.0,Don't waste your money,1391040000,"01 30, 2014"
2,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3.0,OK Palette!,1397779200,"04 18, 2014"
3,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4.0,great quality,1378425600,"09 6, 2013"
4,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2.0,Do not work on my face,1386460800,"12 8, 2013"


In [10]:
beauty_df.drop('rt',axis=1,inplace=True)
beauty_df.drop('reviewtime',axis=1,inplace=True)
beauty_df.drop('helpful',axis=1,inplace=True)
beauty_df.drop('userName',axis=1,inplace=True)





In [11]:
beauty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198503 entries, 0 to 198502
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   userId      198503 non-null  object
 1   productId   198503 non-null  object
 2   reviewtext  198476 non-null  object
 3   ratings     198503 non-null  object
 4   summary     198502 non-null  object
dtypes: object(5)
memory usage: 7.6+ MB


In [12]:
rows,columns=beauty_df.shape
print('Number of rows: ',rows)
print('Number of columns: ',columns)

Number of rows:  198503
Number of columns:  5


In [13]:
beauty_df1=beauty_df.iloc[:50000,0:]

In [14]:
beauty_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userId      50000 non-null  object
 1   productId   50000 non-null  object
 2   reviewtext  49994 non-null  object
 3   ratings     50000 non-null  object
 4   summary     50000 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [28]:
beauty_df1['ratings'].describe().transpose()


count     50000
unique        6
top         5.0
freq      30113
Name: ratings, dtype: object

In [29]:
print('Number of unique users in Raw data = ', beauty_df1['userId'].nunique())
# Number of unique product id  in the data
print('Number of unique product in Raw data = ', beauty_df1['productId'].nunique())

Number of unique users in Raw data =  17461
Number of unique product in Raw data =  3401


In [30]:
most_rated=beauty_df1.groupby('userId').size().sort_values(ascending=False)[:10]
print('Top 10 users based on ratings: \n',most_rated)

Top 10 users based on ratings: 
 userId
A2V5R832QCSOMX    70
A281NPSIMI1C2R    61
A3M174IC0VXOS2    55
ALQGOMOY1F5X9     51
A3KEZLJ59C1JVH    49
AKMEY1BSHSDG7     48
AJGU56YG8G1DQ     45
A1RRMZKOMZ2M7J    43
A2C58G8O40YC7T    41
A132ETQPMHQ585    38
dtype: int64


In [31]:
counts=beauty_df1.userId.value_counts()
beauty_df1_final=beauty_df1[beauty_df1.userId.isin(counts[counts>=15].index)]
print('Number of users who have rated 25 or more items =', len(beauty_df1_final))
print('Number of unique users in the final data = ', beauty_df1_final['userId'].nunique())
print('Number of unique products in the final data = ', beauty_df1_final['userId'].nunique())

Number of users who have rated 25 or more items = 2212
Number of unique users in the final data =  98
Number of unique products in the final data =  98


In [32]:
final_ratings_matrix = beauty_df1_final.pivot(index = 'userId', columns ='productId', values = 'ratings').fillna(0)
final_ratings_matrix.head()

productId,9788072216,9790794231,B00004TMFE,B00004U9UY,B000050B6U,B000052WYD,B000052WYL,B000052XIA,B000052Y33,B000052YJM,...,B001AH8PZY,B001AHAEZ8,B001ALPKXA,B001AMOKR6,B001AN0UTC,B001ANOONK,B001AO0WCG,B001AQ6HDM,B001ARQWW2,B001ARSVI0
userId,,,,,,,,,,,,,,,,,,,,,
A132ETQPMHQ585,0,0,0,0,0,0,0,5.0,0,0,...,0,0,0,0,0,0,0,0,0,0
A14E70DKB6QPZY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A18Z7ICRLVHBBP,0,0,0,0,0,5.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A197BFZ8BSVSO2,0,0,0,0,0,0,0,0,0,0,...,0,0,5.0,0,0,0,0,0,0,0
A19UTUEBWKIZFT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train_data, test_data = train_test_split(beauty_df1_final, test_size = 0.3, random_state=0)
train_data.head()

,userId,productId,reviewtext,ratings,summary
19737,A1IU7S4HCK1XK0,B000FL4410,"This shampoo, which we found at our local groc...",3.0,"No sulfates, volumizing, but leaves hair stick..."
14144,ALQGOMOY1F5X9,B0009WY4U6,I have very dry mature skin and this cleanser ...,1.0,st ives cleanser
28351,A197BFZ8BSVSO2,B000P22TIY,This cologne smells great i love the bottle it...,5.0,Great Cologne
38961,A89ZZKNDFQH23,B000ZMBSPE,I used this with apple cider vinegar. I used ...,4.0,"hmmm..not a miracle, but above average"
6243,A14E70DKB6QPZY,B00016XJ4M,"I am an admitted product junkie, but it's back...",5.0,Simplest Cure for Oily Skin


In [34]:
print('Shape of training data: ',train_data.shape)
print('Shape of testing data: ',test_data.shape)

Shape of training data:  (1548, 5)
Shape of testing data:  (664, 5)


In [35]:
train_data_grouped = train_data.groupby('productId').agg({'userId': 'count'}).reset_index()
train_data_grouped.rename(columns = {'userId': 'score'},inplace=True)
train_data_grouped.head(40)

,productId,score
0,9788072216,1
1,B00004TMFE,1
2,B00004U9UY,1
3,B000050B6U,1
4,B000052WYD,1
5,B000052WYL,1
6,B000052YJM,2
7,B000052YM4,1
8,B000052YM7,6
9,B000052YM8,2


In [36]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'productId'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations

,productId,score,rank
148,B00021B8L2,8,1.0
111,B000142FVW,7,2.0
733,B000TKH6G2,7,3.0
8,B000052YM7,6,4.0
150,B00021C1LI,6,5.0


In [37]:
def recommend(user_id):     
    user_recommendations = popularity_recommendations 
          
    #Add user_id column for which the recommendations are being generated 
    user_recommendations['userId'] = user_id 
      
    #Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations

In [40]:
find_recom = [100,101,102]   # This list is user choice.
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n")

The list of recommendations for the userId: 100

     userId   productId  score  rank
148     100  B00021B8L2      8   1.0
111     100  B000142FVW      7   2.0
733     100  B000TKH6G2      7   3.0
8       100  B000052YM7      6   4.0
150     100  B00021C1LI      6   5.0


The list of recommendations for the userId: 101

     userId   productId  score  rank
148     101  B00021B8L2      8   1.0
111     101  B000142FVW      7   2.0
733     101  B000TKH6G2      7   3.0
8       101  B000052YM7      6   4.0
150     101  B00021C1LI      6   5.0


The list of recommendations for the userId: 102

     userId   productId  score  rank
148     102  B00021B8L2      8   1.0
111     102  B000142FVW      7   2.0
733     102  B000TKH6G2      7   3.0
8       102  B000052YM7      6   4.0
150     102  B00021C1LI      6   5.0


